In [1]:
import sqlite3 as lite
import pandas as pd

con = lite.connect('./data/checking-logs.sqlite') # подключение БД к Питон

In [2]:
datamart = pd.io.sql.read_sql("""
    SELECT c.uid, labname, timestamp AS first_commit_ts, min(datetime) AS first_view_ts 
    FROM checker AS c LEFT JOIN pageviews AS p
    ON c.uid = p.uid
    WHERE status = 'ready' 
                    and numTrials = 1 
                    and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
                    and c.uid LIKE 'user_%'
    GROUP BY  timestamp, c.uid 
    """, con, parse_dates = ['first_commit_ts', 'first_view_ts']) # parse_dates - преобразует в формат датавремя
datamart.to_sql('datamart', con, if_exists = 'replace') # запись новой таблицы (с назв. datamart) в БД; if_exists - перезаписывает табл
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [3]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [4]:
test = datamart.dropna() 
control = datamart[datamart.first_view_ts.isna()].copy()
control.first_view_ts = test.first_view_ts.mean()
test.to_sql('test', con, if_exists = 'replace')
control.to_sql('control', con, if_exists = 'replace')

81

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [6]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [7]:
con.close()